In [19]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds



In [25]:
mnist_dataset, mnist_info= tfds.load(name='mnist', with_info= True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_test_samples = mnist_info.splits ['test'].num_examples

num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale (image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)


In [26]:
Buffer_size = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(Buffer_size)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

Batch_size = 100

train_data = train_data.batch(Batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))

2022-01-14 18:58:45.258418: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-14 18:58:46.788436: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [28]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [29]:
num_epochs = 5
model.fit(train_data, epochs = num_epochs, validation_data=(validation_inputs,validation_targets), verbose=2)

Epoch 1/5
540/540 - 14s - loss: 0.4050 - accuracy: 0.8859 - val_loss: 0.2113 - val_accuracy: 0.9365
Epoch 2/5
540/540 - 9s - loss: 0.1886 - accuracy: 0.9445 - val_loss: 0.1536 - val_accuracy: 0.9523
Epoch 3/5
540/540 - 8s - loss: 0.1450 - accuracy: 0.9573 - val_loss: 0.1201 - val_accuracy: 0.9648
Epoch 4/5
540/540 - 7s - loss: 0.1167 - accuracy: 0.9648 - val_loss: 0.1048 - val_accuracy: 0.9688
Epoch 5/5
540/540 - 6s - loss: 0.0982 - accuracy: 0.9700 - val_loss: 0.0987 - val_accuracy: 0.9740


In [30]:
test_loss,test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}, Test accuracy: {1:.2f}%'.format)

1/1 [==============================] - 1s 1s/step - loss: 0.1179 - accuracy: 0.9628
<built-in method format of str object at 0x7f1a84221330>
